<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# uncomment on google colab, comment on kaggle.

In [1]:
# !mkdir ~/.kaggle
# from google.colab import files
# files.upload()
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d shusrith/betterer-preproc

In [2]:
# !unzip /content/betterer-preproc.zip

In [3]:
# !kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

In [4]:
# !unzip /content/alzheimers-dataset-4-class-of-images.zip

In [5]:
# !rm -rf /kaggle/working/test

# PRE PROCESS
.JPG TO .NPY

In [6]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img = cv2.resize(img, (256, 256)) 
            img_array = np.array(img)
            folder = input_folder.split("/")[-1]
            output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
            np.save(output_file_path, img_array)

input_folder_path = "/kaggle/input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/train"
output_folder_path = "/kaggle/working/train"

jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [7]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img = cv2.resize(img, (256, 256))
            img_array = np.array(img)
            folder = input_folder.split("/")[-1]
            output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
            np.save(output_file_path, img_array)

jpg_to_npy_in_folders("/kaggle/input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/test", "/kaggle/working/test")



In [8]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)


1101

In [9]:
ad = [i for i in files if "AD" in i]
cn = [i for i in files if "CN" in i]
emci = [i for i in files if "EMCI" in i]
lmci = [i for i in files if "LMCI" in i]
mci = [i for i in files if " MCI " in i]
print(len(ad), len(cn), len(emci), len(lmci), len(mci))

145 493 204 61 198


In [10]:
import pandas as pd
l = [[ad, 0], [cn, 1], [emci, 2], [lmci, 3], [mci, 4]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])


In [11]:
toreshape = []
for j, i in enumerate(df["Path"]):
    if np.load(i).shape != (256, 256, 3):
        toreshape.append(j)
toreshape

[]

In [12]:
from imblearn.over_sampling import RandomOverSampler
path = np.array(df["Path"]).reshape(-1, 1)
classes = np.array(df["Class"]).reshape(-1, 1)
ros = RandomOverSampler(random_state=42)
paths, oversampled_labels = ros.fit_resample(path, classes)
df = pd.DataFrame({
    'Path': paths.flatten(),
    'Class': oversampled_labels
})
df


,Path,Class
0,/kaggle/working/train/Final AD JPEG48.npy,0
1,/kaggle/working/train/Final AD JPEG95.npy,0
2,/kaggle/working/train/Final AD JPEG113.npy,0
3,/kaggle/working/train/Final AD JPEG117.npy,0
4,/kaggle/working/train/Final AD JPEG31.npy,0
...,...,...
2460,/kaggle/working/train/Final MCI JPEG89.npy,4
2461,/kaggle/working/train/Final MCI JPEG11.npy,4
2462,/kaggle/working/train/Final MCI JPEG166.npy,4
2463,/kaggle/working/train/Final MCI JPEG123.npy,4


In [13]:
import numpy as np

images = np.array([])  
for j, i in enumerate(df["Path"]):
    loaded_array = np.load(i)
    if j % 50 == 0:
        print(j)
    images = np.append(images, loaded_array)  


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450


In [14]:
x = df["Class"].value_counts()
x

Class
0    493
1    493
2    493
3    493
4    493
Name: count, dtype: int64

In [15]:
images = images.reshape((2465, 256, 256, 3))

In [16]:
from sklearn.model_selection import train_test_split
labels = np.array(df["Class"])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [17]:
print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

(1972, 256, 256, 3) (493, 256, 256, 3) (1972,) (493,)


In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np

input_shape = train_images[0].shape

train_labels_encoded = to_categorical(train_labels, num_classes=5)
test_labels_encoded = to_categorical(test_labels, num_classes=5)


model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))
model.summary()

2024-04-22 08:43:30.835619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 08:43:30.835750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 08:43:30.994809: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 230400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │   117,965,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,402,437 (451.67 MB)

 Trainable params: 118,402,437 (451.67 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
from keras.optimizers import Adam
def lr_schedule(epoch, learning_rate):
    if epoch < 20:
        return learning_rate
    else:
        if epoch % 2 == 0:
          return learning_rate * 0.95
        else:
          return learning_rate

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

In [20]:
import math
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_images, train_labels_encoded,
                    epochs=50,
                    validation_data=(test_images, test_labels_encoded),
                    callbacks=[lr_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
 1/62 ━━━━━━━━━━━━━━━━━━━━ 28:57 28s/step - accuracy: 0.2188 - loss: 12.0945

I0000 00:00:1713775454.801548      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713775454.820094      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.2280 - loss: 74.7283

W0000 00:00:1713775484.141168      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1713775485.998969      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 64s 588ms/step - accuracy: 0.2287 - loss: 73.8983 - val_accuracy: 0.3996 - val_loss: 1.4230 - learning_rate: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50


W0000 00:00:1713775490.866751      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 151ms/step - accuracy: 0.4129 - loss: 1.3965 - val_accuracy: 0.5112 - val_loss: 1.2321 - learning_rate: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 152ms/step - accuracy: 0.5524 - loss: 1.1581 - val_accuracy: 0.6329 - val_loss: 1.0683 - learning_rate: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 152ms/step - accuracy: 0.6925 - loss: 0.8524 - val_accuracy: 0.7383 - val_loss: 0.7086 - learning_rate: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 153ms/step - accuracy: 0.7610 - loss: 0.6522 - val_accuracy: 0.7769 - val_loss: 0.5993 - learning_rate: 0.0010

Epoch 6: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 154ms/step - accuracy: 0.8382 - loss: 0.

In [21]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(test_images)
roc_auc = roc_auc_score(test_labels_encoded, y_prob, average='macro')

print("ROC AUC:", roc_auc)

 5/16 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

W0000 00:00:1713775965.861015      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
ROC AUC: 0.9796250652085263


W0000 00:00:1713775966.696957      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [22]:
model.save("my_cnn_model.h5")


In [23]:
import os
import numpy as np
directory = '/kaggle/working/test'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

195

In [24]:
ad = [i for i in files if "AD" in i]
cn = [i for i in files if "CN" in i]
emci = [i for i in files if "EMCI" in i]
lmci = [i for i in files if "LMCI" in i]
mci = [i for i in files if " MCI " in i]
print(len(ad), len(cn), len(emci), len(lmci), len(mci))

26 87 36 11 35


In [25]:
import pandas as pd
l = [[ad, 0], [cn, 1], [emci, 2], [lmci, 3], [mci, 4]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
1    87
2    36
4    35
0    26
3    11
Name: count, dtype: int64

In [26]:
for j, i in enumerate(df["Path"]):
    if np.load(i).shape != (256, 256, 3):
        df.drop(index=j, inplace=True)

In [27]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [28]:
labels = df["Class"]
labels = to_categorical(labels, num_classes=5)
# Verify the encoded labels
print(labels.shape)

(195, 5)


In [29]:
l = []
for i in labels:
  for j in range(5):
    if i[j] == 1:
      l.append(j)


In [30]:
y_prob = model.predict(images)

y_prob = np.argmax(y_prob, axis=1)  

5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

W0000 00:00:1713775975.275670      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 388ms/step


W0000 00:00:1713775977.599999      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [31]:
from sklearn.metrics import classification_report

print(classification_report(y_prob, l, target_names=["Final AD JPEG" , "Final CN JPEG",  "Final EMCI JPEG","Final LMCI JPEG","Final MCI JPEG"]))

                 precision    recall  f1-score   support

  Final AD JPEG       0.96      1.00      0.98        25
  Final CN JPEG       0.97      0.98      0.97        86
Final EMCI JPEG       0.97      0.97      0.97        36
Final LMCI JPEG       1.00      0.92      0.96        12
 Final MCI JPEG       1.00      0.97      0.99        36

       accuracy                           0.97       195
      macro avg       0.98      0.97      0.97       195
   weighted avg       0.97      0.97      0.97       195



In [32]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(l, y_prob))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.96      1.00      0.98      0.98      0.96        26
          1       0.98      0.97      0.98      0.97      0.97      0.95        87
          2       0.97      0.97      0.99      0.97      0.98      0.96        36
          3       0.92      1.00      0.99      0.96      1.00      1.00        11
          4       0.97      1.00      0.99      0.99      1.00      0.99        35

avg / total       0.97      0.97      0.99      0.97      0.98      0.96       195



In [33]:
model.evaluate(images, labels)

5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9667 - loss: 0.1373

W0000 00:00:1713775980.069625      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.9695 - loss: 0.1314


W0000 00:00:1713775980.677918      75 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


[0.12097194045782089, 0.9743589758872986]